In [1]:
import pandas as pd
import eikon as ek
import datetime as dt
from dateutil.relativedelta import relativedelta

ek.set_app_key('89915a3b58874e1599870c6ecc45d6edd6344f8c')

2023-03-10 09:47:06,804 P[20808] [MainThread 21228] Error: no proxy address identified.
Check if Eikon Desktop or Eikon API Proxy is running.
2023-03-10 09:47:06,806 P[20808] [MainThread 21228] Error on handshake url http://127.0.0.1:None/api/handshake : UnsupportedProtocol("Request URL is missing an 'http://' or 'https://' protocol.")
2023-03-10 09:47:06,807 P[20808] [MainThread 21228] Error on handshake url http://127.0.0.1:None/api/handshake : UnsupportedProtocol("Request URL is missing an 'http://' or 'https://' protocol.")
2023-03-10 09:47:06,808 P[20808] [MainThread 21228] Port number was not identified, cannot send any request


In [3]:
fields = ['TR.IndexJLConstituentRIC','TR.IndexJLConstituentRIC.date','TR.IndexJLConstituentRIC.change']
jl_df, err = ek.get_data('.STOXX',fields,parameters={'SDate':'2000-01-01','EDate':'2023-01-01','IC':'B'})
c_df, err = ek.get_data('.STOXX','TR.IndexConstituentRIC')

In [21]:
jl_df

,Instrument,Constituent RIC,Date,Change
0,.STOXX,INAI.MI^L01,2000-01-20T00:00:00Z,Leaver
1,.STOXX,MTA1V.HE^C00,2000-01-31T00:00:00Z,Leaver
2,.STOXX,ARG.MC^A00,2000-01-31T00:00:00Z,Leaver
3,.STOXX,VDFGn.DE^H02,2000-02-14T00:00:00Z,Leaver
4,.STOXX,TARM.L^E00,2000-03-08T00:00:00Z,Leaver
...,...,...,...,...
2738,.STOXX,VRLA.PA,2022-12-19T00:00:00Z,Joiner
2739,.STOXX,CMBN.S,2022-12-19T00:00:00Z,Joiner
2740,.STOXX,P911_p.DE,2022-12-19T00:00:00Z,Joiner
2741,.STOXX,SGREN.MC^B23,2022-12-23T00:00:00Z,Leaver


In [22]:
c_df

,Instrument,Constituent RIC
0,.STOXX,BOUY.PA
1,.STOXX,AMPF.MI
2,.STOXX,GETP.PA
3,.STOXX,HRGV.L
4,.STOXX,INGA.AS
...,...,...
595,.STOXX,TOP.CO
596,.STOXX,MBGn.DE
597,.STOXX,NG.L
598,.STOXX,SWECb.ST


In [13]:
current_constituents = c_df['Constituent RIC'].tolist()

In [14]:
start_date = dt.date(2000,1,1)
end_date = dt.date(2023,1,1)
dates = [start_date]
i = 0
while (date := start_date + relativedelta(days=i+1)) <= end_date:
    dates.append(date)
    i +=1

In [18]:
jl_df_copy = jl_df.copy()

In [87]:
jl_df = jl_df_copy.copy()

In [88]:
jl_df['Date'] = pd.to_datetime(jl_df.Date).dt.date

In [197]:
jl_df.sort_values(['Date','Change'],ascending=False,inplace=True)
jl_df

,Change,Company,Code,Date,Constituent RIC
3851,Leaver,Linde IR,LIN.DE,2023-02-27,LIN.DE
3852,Joiner,Sinch,SINCH.ST,2023-02-27,SINCH.ST
3849,Leaver,AVEVA Group,AVV.L^A23 (expired),2023-01-18,AVV.L^A23
3850,Joiner,Brunello,BCU.MI,2023-01-18,BCU.MI
3848,Leaver,Homeserve,HSV.L^A23 (expired),2023-01-04,HSV.L^A23
...,...,...,...,...,...
634,Joiner,Woolwich,WWH.L^J00 (expired),1998-08-12,WWH.L^J00
635,Joiner,WPP,WPP.L,1998-08-12,WPP.L
636,Joiner,Zodiac Aero,ZODC.PA^C18 (expired),1998-08-12,ZODC.PA^C18
637,Joiner,Zurich Ins Group,ZURN.S,1998-08-12,ZURN.S


In [90]:
#jl_df = jl_df.set_index('Date')

In [41]:
for date, row in jl_df.loc[dt.date(2022,12,23)].T.items():
    print(row['Change'] == 'Leaver', row['Constituent RIC'])

True SGREN.MC^B23
False TLXGn.DE


In [198]:
dates_dict = {}
current_constituents = c_df['Constituent RIC'].tolist()
for date in dates[::-1]:
    if date in jl_df.Date.values:
        for _, row in jl_df.loc[jl_df.Date == date].T.items():
            value = row['Constituent RIC']
            if row['Change'] == 'Leaver':
                current_constituents.append(value)
            else:
                try:
                    current_constituents.remove(row['Constituent RIC'])
                except ValueError:
                    print(f'{value} is missing')
    dates_dict[date] = set(current_constituents)

AVV.L^A23 is missing
LIN.DE is missing
HSV.L^A23 is missing


In [205]:
new_dates_dict = {str(key):list(value) for key,value in dates_dict.items()}

In [203]:
new_dates_dict.keys()

dict_keys(['2023-01-01', '2022-12-31', '2022-12-30', '2022-12-29', '2022-12-28', '2022-12-27', '2022-12-26', '2022-12-25', '2022-12-24', '2022-12-23', '2022-12-22', '2022-12-21', '2022-12-20', '2022-12-19', '2022-12-18', '2022-12-17', '2022-12-16', '2022-12-15', '2022-12-14', '2022-12-13', '2022-12-12', '2022-12-11', '2022-12-10', '2022-12-09', '2022-12-08', '2022-12-07', '2022-12-06', '2022-12-05', '2022-12-04', '2022-12-03', '2022-12-02', '2022-12-01', '2022-11-30', '2022-11-29', '2022-11-28', '2022-11-27', '2022-11-26', '2022-11-25', '2022-11-24', '2022-11-23', '2022-11-22', '2022-11-21', '2022-11-20', '2022-11-19', '2022-11-18', '2022-11-17', '2022-11-16', '2022-11-15', '2022-11-14', '2022-11-13', '2022-11-12', '2022-11-11', '2022-11-10', '2022-11-09', '2022-11-08', '2022-11-07', '2022-11-06', '2022-11-05', '2022-11-04', '2022-11-03', '2022-11-02', '2022-11-01', '2022-10-31', '2022-10-30', '2022-10-29', '2022-10-28', '2022-10-27', '2022-10-26', '2022-10-25', '2022-10-24', '2022-10-

In [206]:
import json

with open('daily_constituents_filter.json','w') as file:
    json.dump(new_dates_dict,file)

In [117]:
current_constituents

In [122]:
set(dates_dict[dt.date(2022,12,23)]) - set(dates_dict[dt.date(2022,12,24)])

{'SGREN.MC^B23'}

In [106]:
len(dates_dict[dt.date(2022,12,23)])

601

In [162]:
for i in dates_dict.values():
    print(len(i))

600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
600
601
600
600
600
600
600
600
600
600
600
600
600
600
600
600
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
600
600
600
600
600
600
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
600
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
599
600
600
600
599
599
599
599
599
599
599
599
599
599


In [125]:
pd.DataFrame(dates_dict)

ValueError: All arrays must be of the same length

In [79]:
for _, row in jl_df.loc[dt.date(2022,10,4)].T.items():
    print(type(row))

<class 'str'>
<class 'str'>
<class 'str'>


In [86]:
jl_df.loc[jl_df.index == dt.date(2022,10,4)]

,Instrument,Constituent RIC,Change
Date,,,
2022-10-04,.STOXX,ACLN.S,Leaver


In [84]:
list(jl_df.loc[dt.date(2022,12,19)]..items())

[(datetime.date(2022, 12, 19),
  Instrument          .STOXX
  Constituent RIC    VITR.ST
  Change              Leaver
  Name: 2022-12-19, dtype: string),
 (datetime.date(2022, 12, 19),
  Instrument          .STOXX
  Constituent RIC    ADEA.OL
  Change              Leaver
  Name: 2022-12-19, dtype: string),
 (datetime.date(2022, 12, 19),
  Instrument         .STOXX
  Constituent RIC    DARK.L
  Change             Leaver
  Name: 2022-12-19, dtype: string),
 (datetime.date(2022, 12, 19),
  Instrument          .STOXX
  Constituent RIC    PROX.BR
  Change              Leaver
  Name: 2022-12-19, dtype: string),
 (datetime.date(2022, 12, 19),
  Instrument            .STOXX
  Constituent RIC    VPLAYb.ST
  Change                Leaver
  Name: 2022-12-19, dtype: string),
 (datetime.date(2022, 12, 19),
  Instrument         .STOXX
  Constituent RIC    BPTB.L
  Change             Leaver
  Name: 2022-12-19, dtype: string),
 (datetime.date(2022, 12, 19),
  Instrument         .STOXX
  Constituent RIC

In [88]:
missing_rics[0]

'INAI.MI^L01'

In [83]:
len(missing_rics)

457

In [96]:
for i in 
assert missing_rics[0] in og_list

AssertionError: 

In [89]:
pd.DataFrame(dates_dict)

,2000-01-01,2000-01-02,2000-01-03,2000-01-04,2000-01-05,2000-01-06,2000-01-07,2000-01-08,2000-01-09,2000-01-10,...,2022-12-23,2022-12-24,2022-12-25,2022-12-26,2022-12-27,2022-12-28,2022-12-29,2022-12-30,2022-12-31,2023-01-01
0,ULVR.L,ULVR.L,ULVR.L,ULVR.L,ULVR.L,ULVR.L,ULVR.L,ULVR.L,ULVR.L,ULVR.L,...,ULVR.L,ULVR.L,ULVR.L,ULVR.L,ULVR.L,ULVR.L,ULVR.L,ULVR.L,ULVR.L,ULVR.L
1,UU.L,UU.L,UU.L,UU.L,UU.L,UU.L,UU.L,UU.L,UU.L,UU.L,...,UU.L,UU.L,UU.L,UU.L,UU.L,UU.L,UU.L,UU.L,UU.L,UU.L
2,UPM.HE,UPM.HE,UPM.HE,UPM.HE,UPM.HE,UPM.HE,UPM.HE,UPM.HE,UPM.HE,UPM.HE,...,UPM.HE,UPM.HE,UPM.HE,UPM.HE,UPM.HE,UPM.HE,UPM.HE,UPM.HE,UPM.HE,UPM.HE
3,VLOF.PA,VLOF.PA,VLOF.PA,VLOF.PA,VLOF.PA,VLOF.PA,VLOF.PA,VLOF.PA,VLOF.PA,VLOF.PA,...,VLOF.PA,VLOF.PA,VLOF.PA,VLOF.PA,VLOF.PA,VLOF.PA,VLOF.PA,VLOF.PA,VLOF.PA,VLOF.PA
4,SGEF.PA,SGEF.PA,SGEF.PA,SGEF.PA,SGEF.PA,SGEF.PA,SGEF.PA,SGEF.PA,SGEF.PA,SGEF.PA,...,SGEF.PA,SGEF.PA,SGEF.PA,SGEF.PA,SGEF.PA,SGEF.PA,SGEF.PA,SGEF.PA,SGEF.PA,SGEF.PA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,P911_p.DE,P911_p.DE,P911_p.DE,P911_p.DE,P911_p.DE,P911_p.DE,P911_p.DE,P911_p.DE,P911_p.DE,P911_p.DE,...,P911_p.DE,P911_p.DE,P911_p.DE,P911_p.DE,P911_p.DE,P911_p.DE,P911_p.DE,P911_p.DE,P911_p.DE,P911_p.DE
425,SUBC.OL,SUBC.OL,SUBC.OL,SUBC.OL,SUBC.OL,SUBC.OL,SUBC.OL,SUBC.OL,SUBC.OL,SUBC.OL,...,SUBC.OL,SUBC.OL,SUBC.OL,SUBC.OL,SUBC.OL,SUBC.OL,SUBC.OL,SUBC.OL,SUBC.OL,SUBC.OL
426,ENOG.L,ENOG.L,ENOG.L,ENOG.L,ENOG.L,ENOG.L,ENOG.L,ENOG.L,ENOG.L,ENOG.L,...,ENOG.L,ENOG.L,ENOG.L,ENOG.L,ENOG.L,ENOG.L,ENOG.L,ENOG.L,ENOG.L,ENOG.L
427,VIS.MC,VIS.MC,VIS.MC,VIS.MC,VIS.MC,VIS.MC,VIS.MC,VIS.MC,VIS.MC,VIS.MC,...,VIS.MC,VIS.MC,VIS.MC,VIS.MC,VIS.MC,VIS.MC,VIS.MC,VIS.MC,VIS.MC,VIS.MC


In [42]:
a = [1,2,3]
a.remove(2)

In [43]:
a

[1, 3]

In [129]:
df, err = ek.get_data('0#.STOXX(2000-01-01)','TR.RIC')

In [152]:
jl_df_og,err = ek.get_data('.STOXX',fields,parameters={'SDate':'1930-01-01','EDate':'2023-01-01','IC':'B'})

In [158]:
og_df, err = ek.get_data('0#.STOXX(1998-08-12)','TR.RIC')

In [159]:
og_df

,Instrument,RIC
0,III.L,III.L
1,POTAb.CO^F05,POTAb.CO^F05
2,ABBZa.S^K99,ABBZa.S^K99
3,ABE.MC^H18,ABE.MC^H18
4,ABF.L,ABF.L
...,...,...
634,WWH.L^J00,WWH.L^J00
635,WPP.L,WPP.L
636,ZODC.PA^C18,ZODC.PA^C18
637,ZURN.S,ZURN.S


In [157]:
jl_df_og.sort_values(['Date','Change']).head(700)

,Instrument,Constituent RIC,Date,Change
0,.STOXX,RBY.L^C00,1998-08-12T00:00:00Z,Joiner
1,.STOXX,NWG.L,1998-08-12T00:00:00Z,Joiner
2,.STOXX,BDEV.L,1998-08-12T00:00:00Z,Joiner
3,.STOXX,DCO.CO^F11,1998-08-12T00:00:00Z,Joiner
4,.STOXX,BAA.L^H06,1998-08-12T00:00:00Z,Joiner
...,...,...,...,...
694,.STOXX,CTX.L^F00,1998-12-21T00:00:00Z,Leaver
695,.STOXX,BDW.L^A03,1998-12-21T00:00:00Z,Leaver
696,.STOXX,VCP.MC^L99,1998-12-21T00:00:00Z,Leaver
697,.STOXX,BAII.PA^A04,1998-12-21T00:00:00Z,Leaver


In [163]:
start_date = dt.date(1998,8,12)
end_date = dt.date(2023,1,1)
dates_1998 = [start_date]
i = 0
while (date := start_date + relativedelta(days=i+1)) <= end_date:
    dates_1998.append(date)
    i +=1

In [160]:
og_list = og_df.RIC.tolist()

In [164]:
jl_df_og['Date'] = pd.to_datetime(jl_df_og.Date).dt.date

In [173]:
jl_df_og = jl_df_og.sort_values(['Date','Change'],ascending=True)
dates_dict_og = {}
current_list = og_list
missing_rics_og = []
for date in dates_1998:
    if date in jl_df_og.Date.values:
        rows = jl_df_og.loc[jl_df_og.Date == date]
        for _, row in rows.T.items():
            value = row['Constituent RIC']
            if row['Change'] == 'Joiner':
                print('adding')
                current_constituents.append(value)
            else:
                try:
                    current_constituents.remove(value)
                except ValueError:
                    #print(f'{value} is missing')
                    missing_rics_og.append(value)
    dates_dict_og[date] = set(current_list)

adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding
adding

In [175]:
jl_df_og

,Instrument,Constituent RIC,Date,Change
0,.STOXX,RBY.L^C00,1998-08-12,Joiner
1,.STOXX,NWG.L,1998-08-12,Joiner
2,.STOXX,BDEV.L,1998-08-12,Joiner
3,.STOXX,DCO.CO^F11,1998-08-12,Joiner
4,.STOXX,BAA.L^H06,1998-08-12,Joiner
...,...,...,...,...
3837,.STOXX,VPLAYb.ST,2022-12-19,Leaver
3838,.STOXX,BPTB.L,2022-12-19,Leaver
3839,.STOXX,DARK.L,2022-12-19,Leaver
3846,.STOXX,TLXGn.DE,2022-12-23,Joiner


In [199]:
for key, value in dates_dict.items():
    print(f'{str(key)}: {len(value)}')

2023-01-01: 600
2022-12-31: 600
2022-12-30: 600
2022-12-29: 600
2022-12-28: 600
2022-12-27: 600
2022-12-26: 600
2022-12-25: 600
2022-12-24: 600
2022-12-23: 600
2022-12-22: 600
2022-12-21: 600
2022-12-20: 600
2022-12-19: 600
2022-12-18: 600
2022-12-17: 600
2022-12-16: 600
2022-12-15: 600
2022-12-14: 600
2022-12-13: 600
2022-12-12: 600
2022-12-11: 600
2022-12-10: 600
2022-12-09: 600
2022-12-08: 600
2022-12-07: 600
2022-12-06: 600
2022-12-05: 600
2022-12-04: 600
2022-12-03: 600
2022-12-02: 600
2022-12-01: 600
2022-11-30: 600
2022-11-29: 600
2022-11-28: 600
2022-11-27: 600
2022-11-26: 600
2022-11-25: 600
2022-11-24: 600
2022-11-23: 600
2022-11-22: 600
2022-11-21: 600
2022-11-20: 600
2022-11-19: 600
2022-11-18: 600
2022-11-17: 600
2022-11-16: 600
2022-11-15: 600
2022-11-14: 600
2022-11-13: 600
2022-11-12: 600
2022-11-11: 600
2022-11-10: 600
2022-11-09: 600
2022-11-08: 600
2022-11-07: 600
2022-11-06: 600
2022-11-05: 600
2022-11-04: 600
2022-11-03: 600
2022-11-02: 600
2022-11-01: 600
2022-10-

In [144]:
len(missing_rics_og)

438

In [3]:
df = pd.read_excel('../../../../Downloads/357315789.xlsx',skiprows=8)

In [4]:
df

,Unnamed: 0,Company,Code,Date
0,+,3i Group,III.L,1998-08-12
1,+,A/S POTAGUA,POTAb.CO^F05 (expired),1998-08-12
2,+,ABB,ABBZa.S^K99 (expired),1998-08-12
3,+,Abertis,ABE.MC^H18 (expired),1998-08-12
4,+,ABF,ABF.L,1998-08-12
...,...,...,...,...
3848,-,Homeserve,HSV.L^A23 (expired),2023-01-04
3849,-,AVEVA Group,AVV.L^A23 (expired),2023-01-18
3850,+,Brunello,BCU.MI,2023-01-18
3851,-,Linde IR,LIN.DE,2023-02-27


In [5]:
df.rename(columns={'Unnamed: 0':'Change'},inplace=True)

In [6]:
df['Change']=df.Change.apply(lambda x: 'Joiner' if x == '+' else 'Leaver',)

In [7]:
df['Constituent RIC'] = df.Code.apply(lambda x: x.strip(' (expired)'))

In [8]:
df['Date'] = pd.to_datetime(df.Date).dt.date

In [9]:
jl_df = df

In [10]:
jl_df

,Change,Company,Code,Date,Constituent RIC
0,Joiner,3i Group,III.L,1998-08-12,III.L
1,Joiner,A/S POTAGUA,POTAb.CO^F05 (expired),1998-08-12,POTAb.CO^F05
2,Joiner,ABB,ABBZa.S^K99 (expired),1998-08-12,ABBZa.S^K99
3,Joiner,Abertis,ABE.MC^H18 (expired),1998-08-12,ABE.MC^H18
4,Joiner,ABF,ABF.L,1998-08-12,ABF.L
...,...,...,...,...,...
3848,Leaver,Homeserve,HSV.L^A23 (expired),2023-01-04,HSV.L^A23
3849,Leaver,AVEVA Group,AVV.L^A23 (expired),2023-01-18,AVV.L^A23
3850,Joiner,Brunello,BCU.MI,2023-01-18,BCU.MI
3851,Leaver,Linde IR,LIN.DE,2023-02-27,LIN.DE


In [2]:
prices_df = pd.read_csv('complete_prices.csv',index_col=0)

In [11]:
df['Constituent RIC'].where(df['Constituent RIC'] in prices_df.columns)

TypeError: unhashable type: 'Series'

In [45]:
filtered_consts = df['Constituent RIC'].apply(lambda x: pd.NA if x not in prices_df.columns else x).notna()

In [47]:
df.loc[filtered_consts]['Constituent RIC'].nunique()

1540

In [50]:
len(prices_df.columns)

1566

In [43]:
long_str = '''BOUY.PA
AMPF.MI
GETP.PA
HRGV.L
INGA.AS
BRBY.L
DIAS.MI
WLN.PA
SCMN.S
ISP.MI
CABK.MC
HSX.L
SCOR.PA
MWDP.PA
UPM.HE
DSMN.AS
BBVA.MC
EDP.LS
INVEb.ST
ENOG.L
SJP.L
AHT.L
ANTO.L
RHMG.DE
BNPP.PA
SPSN.S
ADDTb.ST
DEMANT.CO
AAK.ST
SIEGn.DE
JYSK.CO
TLIT.MI
OCI.AS
HSBA.L
TEL.OL
INVP.L
BC8G.DE
BATS.L
SOGN.PA
OCDO.L
ELIS.PA
PKO.WA
CFR.S
TW.L
MDBI.MI
RUBF.PA
ESSITYb.ST
SDR.L
BAYGn.DE
ICAG.L
DTEGn.DE
BWY.L
WCHG.DE
FPE3_p.DE
IDSI.L
AKZO.AS
ELISA.HE
VIV.PA
ADEN.S
PRY.MI
NESN.S
CRDI.MI
BOLL.PA
ALLN.S
GL9.I
BASFn.DE
VRLA.PA
HEIO.AS
BLND.L
MICP.PA
LIGHT.AS
CMBN.S
KSP.I
DNB.OL
SABE.MC
NEL.OL
DTGGe.DE
AMS.S
UMI.BR
WLSNc.AS
SY1G.DE
VMUK.L
SOON.S
SVT.L
EPED.PA
TEPRF.PA
AKRBP.OL
SAN.MC
REL.L
PGHN.S
OREP.PA
LDOF.MI
BMEB.L
ALFA.ST
KNEBV.HE
EKTAb.ST
DKSH.S
LUNDb.ST
DB1Gn.DE
HNRGn.DE
AAL.L
DANO.PA
ADYEN.AS
HEIN.AS
CLN.S
ALVG.DE
PST.MI
BVI.PA
TPK.L
CAPP.PA
RIO.L
OSBO.L
EUFI.PA
RYA.I
COFB.BR
SALM.OL
CTEC.L
VCTX.L
EVOG.ST
RAAG.DE
RECI.MI
MT.AS
PEO.WA
EURA.PA
GF.S
FLTRF.I
RCOP.PA
NEXS.PA
ANA.MC
BVIC.L
CAST.ST
CVO.PA
IHG.L
PNDORA.CO
ADML.L
PSN.L
SAABb.ST
TEL2b.ST
ORSTED.CO
MUVGn.DE
ZURN.S
KOJAMO.HE
SWEDa.ST
HEIG.DE
IG.MI
LLOY.L
AIRP.PA
GSK.L
BAES.L
SAVE.ST
BOL.ST
UU.L
RXL.PA
NXT.L
LISP.S
ORK.OL
MOCORP.HE
STB.OL
AV.L
LEGn.DE
TIGOsdb.ST
LIN.DE
NWG.L
VOLVb.ST
HUSQb.ST
SUBC.OL
LOGN.S
VIS.MC
STAN.L
ALSO.PA
DLN.L
TELIA.ST
SOF.BR
IGG.L
GALP.LS
HLMA.L
OMVV.VI
BIRG.I
ACS.MC
VALMT.HE
BESI.AS
IMB.L
HAYS.L
GFCP.PA
RACE.MI
RILBA.CO
ALCC.S
AFXG.DE
BKGH.L
BALN.S
MRCG.DE
SGSN.S
PHNX.L
FBK.MI
BT.L
ROR.L
TSCO.L
DANSKE.CO
PERP.PA
EQTAB.ST
LPPP.WA
SPIE.PA
GTT.PA
HPOLb.ST
DGE.L
ELE.MC
NAFG.DE
VTWRn.DE
BALF.L
HLN.L
PRTP.PA
TRELb.ST
BYG.L
WOSG.L
RKT.L
SAMPO.HE
DNP.WA
AZN.L
KYGa.I
SBBb.ST
CCH.L
RWEG.DE
AMA.MC
ORAN.PA
LXIL.L
WIHL.ST
VWS.CO
LHAG.DE
VERB.VI
AXAF.PA
CNA.L
COLOb.CO
BAVA.CO
SATG_p.DE
BEAN.S
BDEV.L
UHR.S
NHY.OL
WDPP.BR
SAGAb.ST
SSABb.ST
FREG.DE
ASSAb.ST
REY.MI
TEF.MC
VTYV.L
CNHI.MI
FMEG.DE
PUBP.PA
ORNBV.HE
DCC.L
LMPL.L
EXOR.AS
LIFCOb.ST
NEXII.MI
ELUXb.ST
HRA.MI
AIBG.I
GEBN.S
GAW.L
TENR.MI
SDFGn.DE
SECTb.ST
ULVR.L
MONC.MI
SOPR.PA
KINDsdb.ST
AMUN.PA
ADP.PA
BP.L
AT1.DE
ABNd.AS
AEGN.AS
IMI.L
WRT1V.HE
WISEa.L
BAWG.VI
INF.L
FNOX.ST
BNZL.L
EDPR.LS
LSEG.L
KINVb.ST
TECN.S
RAND.AS
UBIP.PA
MRL.MC
KESKOB.HE
ABDN.L
TLXGn.DE
LAND.L
AXFO.ST
INDUc.ST
FER.MC
TUIT.L
SCHP.S
ERICb.ST
LONN.S
NDAFI.HE
JDEP.AS
HFGG.DE
LOIM.PA
FORTUM.HE
CONG.DE
ANE.MC
SFZN.S
IETB.BR
ABBN.S
AIR.PA
IFXGn.DE
BARN.S
CAGR.PA
THULE.ST
EDF.PA
ATCOa.ST
QIA.DE
ZALG.DE
ABF.L
SGRO.L
WEIR.L
SAND.ST
GBLB.BR
RS1R.L
MRON.L
AUTOA.L
BMWG.DE
WBSV.VI
BANB.S
CBRO.L
SRG.MI
RMV.L
IPN.PA
HWDN.L
COL.MC
HMb.ST
VOWG_p.DE
EDEN.PA
SASY.PA
ROG.S
AMBUb.CO
ACKB.BR
JD.L
ALKb.CO
FHZN.S
PHP.L
ENT.L
ITX.MC
PZU.WA
BKT.MC
EVTG.DE
SBRY.L
AD.AS
AKE.PA
RTO.L
CLNX.MC
UMG.AS
CHRH.CO
G24n.DE
NESTE.HE
CBKG.DE
CARR.PA
SIKA.S
WALLb.ST
PNN.L
ITPG.MI
SEBF.PA
SAFE.L
ONT.L
URW.AS
TKWY.AS
BCVN.S
TEMN.S
INDV.L
SKG.I
1COV.DE
VIE.PA
FNTGn.DE
PSPN.S
ACCP.PA
A2.MI
ENR1n.DE
STDM.PA
KPN.AS
ROCKb.CO
WPP.L
ADSGn.DE
LVMH.PA
FERG.L
AENA.MC
HUH1V.HE
MAERSKb.CO
ASRNL.AS
KBC.BR
NTGY.MC
SCTS.L
SPX.L
ABI.BR
DPH.L
DRX.L
WTB.L
AM.PA
DHER.DE
BILL.ST
KNIN.S
FABG.ST
GRG.L
SGE.L
GNS.L
SHLG.DE
CPRI.MI
BEZG.L
PHG.AS
CSGN.S
TRN.MI
HRMS.PA
HSV.L^A23
III.L
MNDI.L
SGOB.PA
SCAb.ST
SMIN.L
INCH.L
BKWB.S
UTG.L
TCFP.PA
YAR.OL
SGEF.PA
ARDS.AS
SIGNC.S
REP.MC
STMN.S
UBSG.S
MOWI.OL
FOUG.PA
TOM.OL
SKFb.ST
KBX.DE
GETIb.ST
HTRO.ST
O2Dn.DE
AVV.L^A23
STLAM.MI
DSV.CO
ESLX.PA
EXPN.L
TTEF.PA
SOBIV.ST
MTXGn.DE
INDT.ST
DAST.PA
HELN.S
ICP.L
QQ.L
EMG.L
RR.L
GLEN.L
SCHN.PA
SKAb.ST
SECUb.ST
HIK.L
ECVG.DE
DLGD.L
LOTB.BR
REDE.MC
SN.L
VACN.S
LEGD.PA
HNKG_p.DE
BOSSn.DE
GRLS.MC
NETCG.CO
ERST.VI
DUFN.S
LGEN.L
ENAG.MC
ASMI.AS
SXS.L
INWT.MI
BBOXT.L
STERV.HE
ASML.AS
TATE.L
G1AG.DE
P911_p.DE
RBREW.CO
LXSG.DE
ARGX.BR
AGRP.L
SRENH.S
EONGn.DE
SHBa.ST
ELI.BR
ITRK.L
ALLFG.AS
SEBa.ST
NEKG.DE
UCB.BR
GALE.S
MKS.L
EQNR.OL
BNRGn.DE
CRDA.L
PRU.L
LTEN.PA
IMCD.AS
ENX.PA
VOLCARb.ST
CRH.I
EXHO.PA
ALFEN.AS
STM.MI
TKAG.DE
FDJ.PA
GASI.MI
TRYG.CO
JMT.LS
SRP.L
VLOF.PA
ISS.CO
NOKIA.HE
EVDG.DE
VOD.L
ENEI.MI
NIBEb.ST
JMAT.L
VOES.VI
RENA.PA
DPWGn.DE
GFTU_u.L
TIETO.HE
MNG.L
PSHG_p.DE
PSON.L
DIOR.PA
CCC.L
ITV.L
CARLb.CO
ENGIE.PA
BEIJb.ST
NZYMb.CO
EMSN.S
AOO.BR
GIVN.S
KOG.OL
PUMG.DE
HOLMb.ST
SMDS.L
BAER.S
HBR.L
KGX.DE
AVANZ.ST
GMAB.CO
SAF.PA
SAPG.DE
SLHN.S
BEIG.DE
BAMI.MI
DPLM.L
SESFd.PA
AIXGn.DE
PKN.WA
EPIRa.ST
GJFG.OL
GN.CO
CPG.L
HOLN.S
NOD.OL
BUCN.S
BAKKA.OL
SHEL.AS
SOIT.PA
NN.AS
IBE.MC
SOLB.BR
FUTR.L
SSE.L
SIM.CO
LATOb.ST
AGES.BR
ALEP.WA
AALB.AS
DBKGn.DE
KGH.WA
NOVN.S
BARC.L
HEXAb.ST
PRX.AS
NOVOb.CO
BALDb.ST
KGF.L
EVKn.DE
BIOX.PA
VNAn.DE
UTDI.DE
ANDR.VI
TOP.CO
MBGn.DE
NG.L
SWECb.ST
ENI.MI
'''.split('\n')

In [51]:
jl_df = df.copy().sort_values(['Date','Change'], ascending=False)

In [52]:
jl_df

,Change,Company,Code,Date,Constituent RIC
3851,Leaver,Linde IR,LIN.DE,2023-02-27,LIN.DE
3852,Joiner,Sinch,SINCH.ST,2023-02-27,SINCH.ST
3849,Leaver,AVEVA Group,AVV.L^A23 (expired),2023-01-18,AVV.L^A23
3850,Joiner,Brunello,BCU.MI,2023-01-18,BCU.MI
3848,Leaver,Homeserve,HSV.L^A23 (expired),2023-01-04,HSV.L^A23
...,...,...,...,...,...
634,Joiner,Woolwich,WWH.L^J00 (expired),1998-08-12,WWH.L^J00
635,Joiner,WPP,WPP.L,1998-08-12,WPP.L
636,Joiner,Zodiac Aero,ZODC.PA^C18 (expired),1998-08-12,ZODC.PA^C18
637,Joiner,Zurich Ins Group,ZURN.S,1998-08-12,ZURN.S


In [38]:
start_date = dt.date(2000,1,1)
end_date = dt.date(2023,1,1)
dates = [start_date]
i = 0
while (date := start_date + relativedelta(days=i+1)) <= end_date:
    dates.append(date)
    i +=1

In [53]:
dates_dict = {}
current_constituents = long_str[:]
for date in dates[::-1]:
    if date in jl_df.Date.values:
        for _, row in jl_df.loc[jl_df.Date == date].T.items():
            value = row['Constituent RIC']
            if row['Change'] == 'Leaver':
                current_constituents.append(value)
            else:
                try:
                    current_constituents.remove(row['Constituent RIC'])
                except ValueError:
                    print(f'{value} is missing')
    dates_dict[date] = set(current_constituents) & set(prices_df.columns)

In [44]:
len(long_str)

601

In [54]:
for key, value in dates_dict.items():
    print(f'{str(key)}: {len(value)}')

2023-01-01: 597
2022-12-31: 597
2022-12-30: 597
2022-12-29: 597
2022-12-28: 597
2022-12-27: 597
2022-12-26: 597
2022-12-25: 597
2022-12-24: 597
2022-12-23: 596
2022-12-22: 596
2022-12-21: 596
2022-12-20: 596
2022-12-19: 596
2022-12-18: 596
2022-12-17: 596
2022-12-16: 596
2022-12-15: 596
2022-12-14: 596
2022-12-13: 596
2022-12-12: 596
2022-12-11: 596
2022-12-10: 596
2022-12-09: 596
2022-12-08: 596
2022-12-07: 596
2022-12-06: 596
2022-12-05: 596
2022-12-04: 596
2022-12-03: 596
2022-12-02: 596
2022-12-01: 596
2022-11-30: 596
2022-11-29: 596
2022-11-28: 596
2022-11-27: 596
2022-11-26: 596
2022-11-25: 596
2022-11-24: 596
2022-11-23: 596
2022-11-22: 596
2022-11-21: 596
2022-11-20: 596
2022-11-19: 596
2022-11-18: 596
2022-11-17: 596
2022-11-16: 596
2022-11-15: 596
2022-11-14: 596
2022-11-13: 596
2022-11-12: 596
2022-11-11: 596
2022-11-10: 596
2022-11-09: 596
2022-11-08: 596
2022-11-07: 596
2022-11-06: 596
2022-11-05: 596
2022-11-04: 596
2022-11-03: 596
2022-11-02: 596
2022-11-01: 596
2022-10-

In [35]:
import json

new_dates_dict = {str(key):list(value) for key,value in dates_dict.items()}
with open('tmp_daily_filter.json','w') as file:
    json.dump(new_dates_dict,file)